# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ciudad lazaro cardenas,17.9583,-102.2000,31.42,66,83,5.68,MX,1689890272
1,1,papatowai,-46.5619,169.4708,5.97,69,10,8.75,NZ,1689890273
2,2,hamilton,39.1834,-84.5333,31.06,69,40,4.63,US,1689889590
3,3,margaret river,-33.9500,115.0667,14.67,68,30,8.52,AU,1689890273
4,4,port-aux-francais,-49.3500,70.2167,1.74,51,75,16.94,TF,1689890274


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Check column types to ensure humidity is an int
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [4]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    color = 'City',
    alpha = 0.8
)

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# - Temperature in between 23-30 degrees
# - Wind speeds less than 4.5 m/s
# - Cloudiness <= 20
# - Humidity in between 50-70 %

vacation_cities = city_data_df[(city_data_df['Max Temp'] >= 23) & (city_data_df['Max Temp'] <= 30) & 
                               (city_data_df['Wind Speed'] <= 4.5) & (city_data_df['Cloudiness'] <= 20) & 
                               (city_data_df['Humidity'] >= 50) & (city_data_df['Humidity'] <= 70)]

# Drop any rows with null values
vacation_cities_noNull = vacation_cities.dropna()

# Display sample data
vacation_cities_noNull

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
68,68,bethel,41.3712,-73.4140,28.62,64,0,3.09,US,1689890294
172,172,pirallahi,40.4723,50.3330,25.08,63,0,2.73,AZ,1689890333
379,379,cloquet,46.7216,-92.4594,24.38,57,0,3.60,US,1689890397
533,533,dayr al balah,31.4166,34.3452,27.67,62,0,2.18,PS,1689890479
565,565,gagliano del capo,39.8442,18.3695,27.67,70,0,0.90,IT,1689890489


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_cities_noNull[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
68,bethel,US,41.3712,-73.4140,64,
172,pirallahi,AZ,40.4723,50.3330,63,
379,cloquet,US,46.7216,-92.4594,57,
533,dayr al balah,PS,31.4166,34.3452,62,
565,gagliano del capo,IT,39.8442,18.3695,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 

# Information for params found here: https://apidocs.geoapify.com/docs/places/#about
params = {
    'categories': 'accommodation.hotel', # https://apidocs.geoapify.com/docs/places/#categories
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search \n---------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df['Lat'][index]
    longitude = hotel_df['Lng'][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{longitude},{latitude},{radius}' # Information on how to set the fitler is taken from this "Filters" table from: https://apidocs.geoapify.com/docs/places/#about 
    params['bias'] = f'proximity:{longitude},{latitude}' # Information on how to set the bias is taken from this "Bias" table from: https://apidocs.geoapify.com/docs/places/#about 
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search 
---------------------
bethel - nearest hotel: Hampton Inn Danbury
pirallahi - nearest hotel: No hotel found
cloquet - nearest hotel: AmericInn
dayr al balah - nearest hotel: البيت
gagliano del capo - nearest hotel: Masseria Palane


,City,Country,Lat,Lng,Humidity,Hotel Name
68,bethel,US,41.3712,-73.4140,64,Hampton Inn Danbury
172,pirallahi,AZ,40.4723,50.3330,63,No hotel found
379,cloquet,US,46.7216,-92.4594,57,AmericInn
533,dayr al balah,PS,31.4166,34.3452,62,البيت
565,gagliano del capo,IT,39.8442,18.3695,70,Masseria Palane


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    color = "City",
    alpha = 0.8,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)